# Data Chunking with Dask

In this notebook we demonstrate:

* Xarray + Dask
* NetCDF file Chunks versus Dask Chunks
* chunk shapes

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. Please visit data collection [catalogue](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563) and [CMIP6 terms of use](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html) for more information. 

---

- Authors: NCI Virtual Research Environment Team
- Keywords: CMIP6, Xarray, Dask, Chunks
- Create Date: 2019-June; Update Date: 2020-May
---

### Load the required modules

In [1]:
import xarray as xr
import netCDF4 as nc
import time
%matplotlib inline

### Data

We will use precipitation data update of RCP8.5 based on SSP5 from GFDL-CM4 ensembles in this example. Let's take a look at some of the data:

In [2]:
!ncdump -hst '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc'

netcdf pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231 {
dimensions:
	lat = 180 ;
	bnds = 2 ;
	lon = 288 ;
	time = UNLIMITED ; // (7300 currently)
variables:
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
		lat:standard_name = "latitude" ;
		lat:cell_methods = "time: point" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_DeflateLevel = 2 ;
		lat:_Shuffle = "true" ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:long_name = "latitude bounds" ;
		lat_bnds:units = "degrees_north" ;
		lat_bnds:axis = "Y" ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_DeflateLevel = 2 ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
		lon:standard_name = "longitude" ;
		lon:cell_methods = "time: poin

### Xarray + Dask

Xarray can automatically wrap its data into Dask arrays. This capability turns Xarray into an extremely powerful tool when working with big earth science datasets. 

To see this in action, we will download a fairly large dataset to analyze. We use Xarray's `open_mfdataset` to allow multiple files to be opened simultaneously.

In [3]:
!ls /g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701
path = '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/*'

pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20350101-20541231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20550101-20741231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20750101-20941231.nc
pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20950101-21001231.nc


In [4]:
f_ssp585 = xr.open_mfdataset(path, combine='by_coords')
f_ssp585

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31390)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(7300, 180, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(7300, 288, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(7300, 180, 288), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(7300, 2), meta=np.ndarray>
Attributes:
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               day
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    comment:                <null ref>
    contact:                gfdl.climate.model.info@noaa.gov
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    creation_date:          2019-03-18T15:13:56Z
    data_specs_version:     01.00.27
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    forcing_index:          1
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GF...
    grid:                   atmos data regridded from Cubed-sphere (c96) to 1...
    grid_label:             gr1
    initialization_index:   1
    institution:            National Oceanic and Atmospheric Administration, ...
    institution_id:         NOAA-GFDL
    license:                CMIP6 model data produced by NOAA-GFDL is license...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       GFDL-CM4
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    source:                 GFDL-CM4 (2018): \naerosol: interactive\natmos: G...
    source_id:              GFDL-CM4
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    tracking_id:            hdl:21.14100/ad8c930c-124f-4b82-98b2-98cc4236ba25
    variable_id:            pr
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  60225.0
    parent_time_units:      days since 1850-1-1

<div class="alert alert-warning">
<b>NOTE:</b> the values are not displayed, since that would trigger computation.
</div>

### Chunks

Notice that it says:`pr(time, lat, lon) float32 dask.array<chunksize=(7300, 180, 288), meta=np.ndarray>`. There is now the `chunksize` component. The data array also becomes a Dask array.

The chunking of the array comes from the integration of Dask with Xarray. Dask divides the data array into small pieces called "chunks", with each chunk designed to be small enough to fit into memory. 

The file itself may be already chunked. Filesystem chunking is available in netCDF-4 and HDF5 datasets. The CMIP6 data should all be netCDF-4 and include some form of chunking for each file.

In [5]:
!ncdump -hst '/g/data/oi10/replicas/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-CM4/ssp585/r1i1p1f1/day/pr/gr1/v20180701/pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231.nc'

netcdf pr_day_GFDL-CM4_ssp585_r1i1p1f1_gr1_20150101-20341231 {
dimensions:
	lat = 180 ;
	bnds = 2 ;
	lon = 288 ;
	time = UNLIMITED ; // (7300 currently)
variables:
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
		lat:standard_name = "latitude" ;
		lat:cell_methods = "time: point" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_DeflateLevel = 2 ;
		lat:_Shuffle = "true" ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:long_name = "latitude bounds" ;
		lat_bnds:units = "degrees_north" ;
		lat_bnds:axis = "Y" ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_DeflateLevel = 2 ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
		lon:standard_name = "longitude" ;
		lon:cell_methods = "time: poin

#### In this case the file is chunked such that  `pr:_ChunkSizes = 1, 180, 288 ;`

Here we see that the data is chunked in time, where one chunk is one time-step and all points in lat-lon.

![](images/chunks.png)
image source: https://www.unidata.ucar.edu/blogs/developer/en/entry/chunking_data_why_it_matters

Consider 2 types of data access
1. Accessing a 2D lat-lon slice in time (RHS figure)
2. Accessing a time series at a single lat-lon point (LHS figure)

With the chunking above, the first type of data access only requires access to a single chunk, while the second type needs to access ALL the chunks of the data array regardless. This dataset will be fastest for 2D lat-lon single time-step data access.

In general, even without chunking - when the data is accessed contiguously (by index order) - time is the slowest variable to access, then y, with x being the fastest. With the chunking method of this CMIP6 dataset, time still remains the slowest variable. More uniform variable access speeds would require more evenly spaced chunks.

### Exercise

Time how long it takes to load the precipitation data at `time='2015-01-01'` and then time how long it takes to load the data at `lat=0` and `lon=180` (remember to use `method='nearest'` for the latter case). How much difference in time is there when using these different access methods?


In [6]:
%%time
f_ssp585.pr.sel(time='2015-01-01').load()

CPU times: user 20.7 ms, sys: 12.7 ms, total: 33.4 ms
Wall time: 99.6 ms


<xarray.DataArray 'pr' (time: 1, lat: 180, lon: 288)>
array([[[1.1679080e-07, 1.1679080e-07, 1.1679080e-07, ...,
         1.1679080e-07, 1.1679080e-07, 1.1679080e-07],
        [5.6357038e-08, 5.6899808e-08, 5.7449537e-08, ...,
         5.4770052e-08, 5.5292201e-08, 5.5821182e-08],
        [7.7667160e-08, 7.7346748e-08, 7.7024652e-08, ...,
         7.8618250e-08, 7.8302911e-08, 7.7985881e-08],
        ...,
        [7.0853553e-06, 7.2527428e-06, 7.4210134e-06, ...,
         6.5884897e-06, 6.7532287e-06, 6.9188504e-06],
        [2.2063125e-05, 2.2134822e-05, 2.2207438e-05, ...,
         2.1853495e-05, 2.1922468e-05, 2.1992342e-05],
        [3.0045994e-05, 3.0045994e-05, 3.0045994e-05, ...,
         3.0045994e-05, 3.0045994e-05, 3.0045994e-05]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time     (time) object 2015-01-01 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [7]:
%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load()

CPU times: user 37.4 s, sys: 3.65 s, total: 41 s
Wall time: 42.3 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

### The same volume of data can take orders of magnitude longer to load

The spatial dataset contained 51840 data-points and took in the order of 100ms to load. The time-series dataset had 31390 data-points and took about 10,000 ms to load.

<div class="alert alert-info">
<b>NOTE:</b> If you look at the dashboard, the task stream actually shows that the most time consuming part is data concatenation. 
</div>

Chunking and the ways in which data is read is important when considering both how you access the data and if you wish to parallelise your code.

### NetCDF file Chunks versus Dask Chunks

Keep in mind, Dask chunking is different to chunking of the stored data. As we saw in our example, the stored data was chunked with chunks of size (1,180,288) whereas the Dask array had a chunk size of (7300, 180, 288). It's possible to change the chunking size in the Dask array. In the example below, we are specifying that there are 730 chunks in time.

In [8]:
f_ssp585 = xr.open_mfdataset(path,chunks={'time':730}, combine='by_coords')

f_ssp585

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 31390)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(7300, 180, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(7300, 288, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(730, 180, 288), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(730, 2), meta=np.ndarray>
Attributes:
    external_variables:     areacella
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               day
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    comment:                <null ref>
    contact:                gfdl.climate.model.info@noaa.gov
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    creation_date:          2019-03-18T15:13:56Z
    data_specs_version:     01.00.27
    experiment:             update of RCP8.5 based on SSP5
    experiment_id:          ssp585
    forcing_index:          1
    frequency:              day
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GF...
    grid:                   atmos data regridded from Cubed-sphere (c96) to 1...
    grid_label:             gr1
    initialization_index:   1
    institution:            National Oceanic and Atmospheric Administration, ...
    institution_id:         NOAA-GFDL
    license:                CMIP6 model data produced by NOAA-GFDL is license...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   historical
    parent_mip_era:         CMIP6
    parent_source_id:       GFDL-CM4
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    source:                 GFDL-CM4 (2018): \naerosol: interactive\natmos: G...
    source_id:              GFDL-CM4
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    title:                  NOAA GFDL GFDL-CM4 model output prepared for CMIP...
    tracking_id:            hdl:21.14100/ad8c930c-124f-4b82-98b2-98cc4236ba25
    variable_id:            pr
    variant_info:           N/A
    references:             see further_info_url attribute
    variant_label:          r1i1p1f1
    branch_time_in_parent:  60225.0
    parent_time_units:      days since 1850-1-1

### How big do you make your chunks?

The rule of thumb for Dask chunks is that you should "create arrays with a minimum chunksize of at least one million elements":  http://xarray.pydata.org/en/stable/dask.html#chunking-and-performance

NetCDF4 chunks are often a lot smaller than Dask array chunks. The minimum chunksize exists because if you have too many chunks, queuing of operations when parallelising will be slow. If the chunk sizes are too large, computation and memory can be wasted. The default chunks from dask gave us chunks of size (7300, 180, 288) or nearly 400 million elements so we could try reducing those chunks if needed. The larger the array, the larger the cost of queueing and therefore larger chunks may be needed.

#### IMPORTANT: Whatever Dask array chunks you use, make sure they align with the netCDF4 file chunks!!

So far our chunks have been in time, and the netCDF4 file is also chunked in time. If we tried to use dask chunks to optimise the time-series loading of data, it would not help! 

#### Exercise

Try to load the data in with chunks size `(31390,180,1)` (i.e. chunked in lon) and name that file `f_bad_chunk`. Next, try reloading the time series of pr at `lat=0` and `lon=180` and time how long this takes.

In [9]:
f_bad_chunk = xr.open_mfdataset(path,chunks={'time':31390,'lat':180,'lon':1}, combine='by_coords')

Try running your previous code for `f_bad_chunk` again loading the time series of pr at `lat=0` and `lon=180` and time how long it takes if you scale up or down the number of workers. 

Do the same with the original chunking method of `f_ssp585` and see if there is a difference.

We will use three different schedulers to read the data. First, let's initiate a client.

In [ ]:
# If you run this notebook on your local computer or NCI's VDI instance, you can create cluster
from dask.distributed import Client
client = Client()
print(client)

In [ ]:
# If you run this notebook on Gadi under pangeo environment, you can create cluster using scheduler.json file
from dask.distributed import Client, LocalCluster
client = Client(scheduler_file='../scheduler.json')
print(client)

<div class="alert alert-info">
<b>Warning: Please make sure you specify the correct path to the schedular.json file within your environment.</b>  
</div>

Starting the Dask Client will provide a dashboard which is useful to gain insight into the computation. The link to the dashboard will become visible when you create the Client. We recommend having the Client open on one side of your screen and your notebook open on the other side, which will be useful for learning purposes.

In [11]:
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load(scheduler='synchronous')

CPU times: user 38.1 s, sys: 3.18 s, total: 41.2 s
Wall time: 40.6 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [12]:
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load(scheduler='threads')

CPU times: user 38.5 s, sys: 3.12 s, total: 41.7 s
Wall time: 40.9 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [13]:
%%time
f_bad_chunk.pr.sel(lat=0,lon=180,method='nearest').load(scheduler='processes')

CPU times: user 440 ms, sys: 247 ms, total: 687 ms
Wall time: 14.3 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

Let's switch the scheduler to `threaded scheduler`:

In [14]:
%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load(scheduler='synchronous')

CPU times: user 38.7 s, sys: 2.52 s, total: 41.3 s
Wall time: 40.9 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [15]:
%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load(scheduler='threads')

CPU times: user 39.1 s, sys: 3.04 s, total: 42.1 s
Wall time: 41.6 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

In [16]:
%%time
f_ssp585.pr.sel(lat=0,lon=180,method='nearest').load(scheduler='processes')

CPU times: user 458 ms, sys: 184 ms, total: 643 ms
Wall time: 13.7 s


<xarray.DataArray 'pr' (time: 31390)>
array([2.1776618e-06, 9.6000194e-06, 5.3295435e-06, ..., 7.7904500e-05,
       1.6186357e-05, 1.5706388e-05], dtype=float32)
Coordinates:
    lat      float64 0.5
    lon      float64 180.6
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
Attributes:
    long_name:      Precipitation
    units:          kg m-2 s-1
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    standard_name:  precipitation_flux
    interp_method:  conserve_order1
    original_name:  pr

### Poor chunking with dask can make your performance worse!

As you can see, bad chunks and the alignment of the chunks slow down the I/O performance significantly. They are both  important to keep in mind when creating Dask chunks. 

### Close the client

Before moving on to the next exercise, make sure to close your client or stop this kernel.

In [4]:
client.close()

### Summary

This example shows how to make data chunking with Dask. 

For further information regarding Dask, please see: https://docs.dask.org/en/latest/